In [24]:
import datetime as dt
import pandas as pd
#from pandas.tseries.holiday import USFederalHolidayCalendar
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from fredapi import Fred
import yfinance as yf
from functools import reduce

from price.price_processing import *
from backtest.metric import *
from strategy.optimize.time_series import *

In [2]:
risky_asset = ['GOOGL', 'KO', 'HD', 'INTU', 'PEP', 'NOW', 'TSLA', 'UNH']
bond_asset = ["SHV", "IEF", "TLT", 'IAU', 'SLV', 'VNQ']
canary_asset = ["VWO", "BND"]
#econ_ind = ['UNRATE', 'SP500']
all_tickers = risky_asset + bond_asset + canary_asset

In [3]:
#econ_df = econ_info(econ_ind)
all_ticker_df = get_price(all_tickers, period='1d', interval='1d', start_date='2012-06-29')
all_ticker_df.index.name = "date_time"

all_ticker_df.index.name = "date_time"
price_df = all_ticker_df.loc[:,~all_ticker_df.columns.duplicated()].copy()
price_df.index = pd.to_datetime(price_df.index)
price_df.isna().sum()
price_df#.to_csv("./price_df")

,GOOGL,KO,HD,INTU,PEP,NOW,TSLA,UNH,SHV,IEF,TLT,IAU,SLV,VNQ,VWO,BND
date_time,,,,,,,,,,,,,,,,
2012-06-29,14.516266,28.154278,41.959538,53.784409,52.216080,24.600000,2.086000,49.762318,102.687859,89.848351,97.563095,31.120001,26.650000,43.129215,29.418522,64.217354
2012-07-02,14.526276,28.417135,41.975361,54.083466,52.289967,24.770000,2.026667,47.856892,102.678543,90.265175,98.569748,31.120001,26.730000,43.603817,29.448000,64.352333
2012-07-03,14.710460,28.503557,40.898476,54.509396,52.289967,25.309999,2.044000,46.683010,102.687859,90.040085,97.812233,31.600000,27.490000,43.860882,29.985817,64.260841
2012-07-05,14.912913,28.247896,41.262707,54.300957,51.853958,25.959999,2.082000,47.474117,102.678543,90.281837,98.312042,31.260000,26.879999,43.590626,29.727957,64.443878
2012-07-06,14.664164,28.139872,41.294395,52.602070,51.890919,25.840000,2.066000,47.482616,102.687859,90.598679,99.225761,30.840000,26.299999,43.696091,29.190121,64.504890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-25,95.220001,60.930000,317.260010,401.899994,171.929993,448.769989,144.429993,492.500000,110.220001,99.529999,107.480003,36.939999,21.969999,88.349998,42.849998,74.309998
2023-01-26,97.519997,60.810001,313.809998,412.549988,170.220001,463.070007,160.270004,492.480011,110.250000,99.169998,106.980003,36.590000,21.990000,89.339996,43.180000,74.169998
2023-01-27,99.370003,60.490002,316.690002,422.619995,169.619995,458.859985,177.899994,486.049988,110.260002,99.019997,106.709999,36.549999,21.680000,90.339996,42.889999,74.089996


In [4]:
target_rebal = rebal_dates(price=price_df, period='month')
target_rebal

DatetimeIndex(['2012-06-29', '2012-07-31', '2012-08-31', '2012-09-28',
               '2012-10-31', '2012-11-30', '2012-12-31', '2013-01-31',
               '2013-02-28', '2013-03-28',
               ...
               '2022-04-29', '2022-05-31', '2022-06-30', '2022-07-29',
               '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30',
               '2022-12-30', '2023-01-31'],
              dtype='datetime64[ns]', length=128, freq=None)

In [5]:
rebal_price_df = price_on_rebal(price=price_df, rebal_dates=target_rebal)
rebal_price_df

,GOOGL,KO,HD,INTU,PEP,NOW,TSLA,UNH,SHV,IEF,TLT,IAU,SLV,VNQ,VWO,BND
2012-06-29,14.516266,28.154278,41.959538,53.784409,52.216080,24.600000,2.086000,49.762318,102.687859,89.848351,97.563095,31.120001,26.650000,43.129215,29.418522,64.217354
2012-07-31,15.840090,29.094080,41.318146,52.711102,53.745762,27.000000,1.828000,43.459095,102.687859,91.107231,101.287430,31.440001,27.120001,43.992722,29.477459,65.008324
2012-08-31,17.144394,26.933628,45.167988,53.183510,53.524052,31.100000,1.901333,46.189648,102.706520,91.000336,99.951553,32.959999,30.790001,43.986122,29.551134,65.113068
2012-09-28,18.881380,27.500969,48.049187,53.492405,52.688457,38.680000,1.952000,47.324764,102.698158,90.683479,97.419983,34.540001,33.480000,43.166584,31.120380,65.231033
2012-10-31,17.024525,26.957190,48.853058,54.142784,51.549355,30.650000,1.875333,47.828663,102.690651,90.324295,96.948448,33.520000,31.270000,42.774582,30.948828,65.165054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30,95.650002,55.625496,274.280670,385.809387,162.247925,377.609985,265.250000,503.487946,108.957878,95.221710,101.467430,31.530001,17.500000,79.052689,35.903538,70.680969
2022-10-31,94.510002,59.428520,294.349274,426.637299,180.454361,420.739990,227.539993,553.443970,109.127579,93.837158,95.415474,30.980000,17.620001,81.823532,34.880253,69.862679
2022-11-30,100.989998,63.610001,323.989990,406.767456,184.360001,416.299988,194.699997,546.076660,109.497536,97.226807,102.238693,33.599998,20.430000,86.852455,39.868771,72.423790
2022-12-30,88.230003,63.610001,315.859985,388.434540,180.660004,388.269989,123.180000,530.179993,109.919998,95.779999,99.559998,34.590000,22.020000,82.480003,38.980000,71.839996


In [6]:
##### PART2. MOMENTUM
###2.1 : TOTAL MOMENTUM 
momentum_df = price_df.loc[target_rebal].copy()
momentum_result_df = (12 * momentum_df.pct_change(1))\
                    + (4*momentum_df.pct_change(3)) \
                    + (2*momentum_df.pct_change(6)) \
                    + (1*momentum_df.pct_change(12))

momentum_result_df = momentum_result_df.dropna()
momentum_result_df.head()
display(momentum_result_df)

###2.2 : MOMENTUM SCORE OF CANARY ASSET
canary_signal_series = (momentum_result_df[canary_asset] > 0).sum(axis=1)
canary_signal_series
momentum_result_df['canary_signal'] = canary_signal_series
type(momentum_result_df.index)

,GOOGL,KO,HD,INTU,PEP,NOW,TSLA,UNH,SHV,IEF,TLT,IAU,SLV,VNQ,VWO,BND
2013-06-28,1.566879,0.413168,1.362794,0.365288,1.017517,2.979814,15.282918,1.771942,-0.003028,-0.586195,-0.869691,-2.976806,-3.611526,-0.088096,-1.200009,-0.373148
2013-07-31,1.159104,0.007640,1.504672,1.073306,0.845870,2.961994,18.014296,3.344663,0.004146,-0.392324,-1.033761,-0.138885,-1.665727,-0.026863,-0.504827,-0.141519
2013-08-30,-0.313575,-0.673859,-0.352613,0.365578,-0.318359,3.404855,18.647612,1.468722,0.001186,-0.516704,-0.766263,0.227564,1.760951,-1.254883,-0.994200,-0.285176
2013-09-30,0.757498,-0.275331,0.698639,1.051460,0.161181,3.655857,18.746038,1.251433,0.003521,0.098109,-0.328585,-0.859913,-1.347009,0.271578,0.952188,0.090440
2013-10-31,3.777114,0.504113,0.726174,2.064652,1.066232,3.075061,7.300350,-0.264037,-0.002385,0.012454,-0.128724,-0.482701,-0.001979,0.553892,0.842898,0.085040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30,-2.791416,-1.505768,-0.741039,-1.871707,-0.513874,-3.434704,-0.221420,-0.044858,0.017373,-1.149609,-2.108071,-1.011990,-0.196226,-2.668378,-2.288172,-1.017380
2022-10-31,-1.598159,0.551846,0.651828,0.761697,1.824092,0.501535,-3.462412,1.717167,0.040919,-0.874051,-2.129311,-0.872434,-0.676179,-0.779015,-1.473755,-0.743469
2022-11-30,0.042708,1.326609,1.742398,-1.173347,1.030899,-0.871429,-3.855313,0.541619,0.080684,0.090276,0.033475,1.080111,2.872125,0.227343,1.458098,0.161959
2022-12-30,-2.597934,0.731702,0.417825,-0.878034,0.477117,-1.463863,-8.103257,0.009397,0.113574,-0.409487,-0.941355,0.751775,2.353435,-0.837176,-0.176476,-0.223890


pandas.core.indexes.datetimes.DatetimeIndex

In [7]:
### VAA의 횡적 자산분배 방식 
### 3: VAA 모멘텀 구현 -> factor class의 method에 포함될 부분

def set_weight(series) -> pd.Series:
    signal = series.loc["canary_signal"]
    if signal == 0 :
        target_assets1 = series.loc[bond_asset].sort_values().iloc[-5:].index
        return pd.Series([1/len(target_assets1)] * len(target_assets1),
                        index=target_assets1.tolist())
    elif signal == 1 :
        target_assets1 = series.loc[bond_asset].sort_values().iloc[-5:].index
        target_assets2 = series.loc[risky_asset].sort_values().iloc[-5:].index
        return pd.Series([1/(len(target_assets1) + len(target_assets2))] * (len(target_assets1) + len(target_assets2)),
                        index=target_assets1.tolist() + target_assets2.tolist()) 
    elif signal == 2:
        target_assets1 = series.loc[risky_asset].sort_values().iloc[-5:].index
        return pd.Series([1/len(target_assets1)] * len(target_assets1),
                        index=target_assets1.tolist())

In [8]:
###3.1: check weight
target_weight_df = momentum_result_df.apply(set_weight, axis=1).fillna(0)
target_weight_df#.sum(axis = 1)

,GOOGL,HD,IAU,IEF,INTU,KO,NOW,PEP,SHV,SLV,TLT,TSLA,UNH,VNQ
2013-06-28,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.0,0.2
2013-07-31,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.0,0.2
2013-08-30,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.0,0.0,0.0
2013-09-30,0.2,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.2,0.2,0.0
2013-10-31,0.2,0.0,0.0,0.0,0.2,0.0,0.2,0.2,0.0,0.0,0.0,0.2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.0,0.0,0.0
2022-10-31,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.2
2022-11-30,0.2,0.2,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.2,0.0
2022-12-30,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.2


In [9]:
signal = target_weight_df.replace({0.2:1, 0.1:1})
signal

,GOOGL,HD,IAU,IEF,INTU,KO,NOW,PEP,SHV,SLV,TLT,TSLA,UNH,VNQ
2013-06-28,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2013-07-31,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2013-08-30,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
2013-09-30,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2013-10-31,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
2022-10-31,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2022-11-30,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2022-12-30,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


In [10]:
### 5. portfolio value 
cum_rets = port_cum_rets(calculate_portvals(price_df=price_df, weight_df=target_weight_df, signal_df=signal, long_only=True))
Metric(portfolio=cum_rets).print_report()

Annualized Return: 15.79%
Annualized Volatility: 15.83%
Skewness: -0.19
Kurtosis: 11.99
Max Drawdown: -24.11%
Max Drawdown Duration: 494 days
Annualized Sharp Ratio: 0.74
Annualized Sortino Ratio: -0.15
Annualized Calmar Ratio: 0.65
Annualized VaR: -0.03
Annualized VaR Ratio: 0.02
Annualized CVaR: -0.04
Annualized CVaR Ratio: 0.02
Annualized hit Ratio: 0.55
Annualized GtP Ratio: 0.99


In [11]:
rebal_price_df = rebal_price_df.iloc[12:,]
rebal_price_df

,GOOGL,KO,HD,INTU,PEP,NOW,TSLA,UNH,SHV,IEF,TLT,IAU,SLV,VNQ,VWO,BND
2013-06-28,22.031281,29.696928,62.586231,55.909203,62.236523,40.389999,7.157333,56.614601,102.705559,86.714073,88.126259,23.980000,18.969999,47.015545,29.670256,62.925560
2013-07-31,22.215965,29.674723,63.846516,58.706665,63.568153,43.580002,8.952000,62.986748,102.733513,86.408234,86.136436,25.719999,19.139999,47.439724,29.876776,63.164909
2013-08-30,21.193693,28.267979,60.178768,58.348473,60.668995,46.880001,11.266667,62.027031,102.733513,85.179024,84.981804,27.059999,22.600000,44.128391,28.851818,62.622772
2013-09-30,21.919670,28.249826,61.600002,60.901718,60.925472,51.950001,12.891333,62.147812,102.742844,86.751595,85.535698,25.780001,20.900000,45.664875,30.950792,63.320309
2013-10-31,25.790291,29.510185,63.256725,65.774452,64.443062,54.610001,10.662667,59.240479,102.724251,87.426346,86.759247,25.700001,21.100000,47.729263,32.292793,63.863209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30,95.650002,55.625496,274.280670,385.809387,162.247925,377.609985,265.250000,503.487946,108.957878,95.221710,101.467430,31.530001,17.500000,79.052689,35.903538,70.680969
2022-10-31,94.510002,59.428520,294.349274,426.637299,180.454361,420.739990,227.539993,553.443970,109.127579,93.837158,95.415474,30.980000,17.620001,81.823532,34.880253,69.862679
2022-11-30,100.989998,63.610001,323.989990,406.767456,184.360001,416.299988,194.699997,546.076660,109.497536,97.226807,102.238693,33.599998,20.430000,86.852455,39.868771,72.423790
2022-12-30,88.230003,63.610001,315.859985,388.434540,180.660004,388.269989,123.180000,530.179993,109.919998,95.779999,99.559998,34.590000,22.020000,82.480003,38.980000,71.839996


In [22]:
rets = rebal_price_df.pct_change().fillna(0)
(target_weight_df * rets).dropna(axis=1)

,GOOGL,HD,IAU,IEF,INTU,KO,NOW,PEP,SHV,SLV,TLT,TSLA,UNH,VNQ
2013-06-28,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-07-31,0.000000,0.000000,0.014512,-0.000705,0.000000,-0.000000,0.000000,0.000000,0.000054,0.000000,-0.004516,0.000000,0.000000,0.001804
2013-08-30,-0.000000,-0.000000,0.010420,-0.002845,-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.036155,-0.002681,0.000000,-0.000000,-0.000000
2013-09-30,0.006851,0.000000,-0.000000,0.000000,0.008752,-0.000000,0.021630,0.000000,0.000000,-0.000000,0.000000,0.028840,0.000389,0.000000
2013-10-31,0.035316,0.000000,-0.000000,0.000000,0.016002,0.000000,0.010241,0.011547,-0.000000,0.000000,0.000000,-0.034576,-0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30,-0.000000,-0.000000,-0.005730,-0.009470,-0.000000,-0.000000,-0.000000,-0.000000,0.000056,0.011098,-0.016470,-0.000000,-0.000000,-0.000000
2022-10-31,-0.000000,0.000000,-0.003489,-0.002908,0.000000,0.000000,0.000000,0.000000,0.000311,0.001371,-0.000000,-0.000000,0.000000,0.007010
2022-11-30,0.013713,0.020140,0.000000,0.000000,-0.000000,0.014072,-0.000000,0.004329,0.000000,0.000000,0.000000,-0.000000,-0.002662,0.000000
2022-12-30,-0.000000,-0.000000,0.005893,-0.002976,-0.000000,0.000000,-0.000000,-0.000000,0.000772,0.015565,-0.000000,-0.000000,-0.000000,-0.010069


In [23]:
# TimeSeries의 변수로 들어갈 포트폴리오 리턴을 만든다. 
def port_val(weight_df, rebal_price): 
    
    # 포트폴리오 수익률 데이터프레임
    rets = rebal_price.pct_change().fillna(0)
    port_rets = (weight_df * rets).dropna(axis=1)
    port_rets = port_rets.sum(axis=1)
    
    return port_rets

port_rets = port_val(weight_df=target_weight_df, rebal_price=rebal_price_df)
port_rets

2013-06-28    0.000000
2013-07-31    0.011150
2013-08-30    0.041049
2013-09-30    0.066462
2013-10-31    0.038530
                ...   
2022-09-30   -0.020516
2022-10-31    0.002296
2022-11-30    0.049591
2022-12-30    0.009185
2023-01-31    0.147465
Length: 116, dtype: float64

In [25]:
ew_weights, ew_ew_weight = TimeSeries(port_rets=port_rets, cs_weight=target_weight_df, call_method='ew', param=12).run()
vt_weights, ew_vt_weight = TimeSeries(port_rets=port_rets, cs_weight=target_weight_df, call_method='vt', param=12).run()
cvt_weights, ew_cvt_weight = TimeSeries(port_rets=port_rets, cs_weight=target_weight_df, call_method='cvt', param=12).run()
kl_weights, ew_kl_weight = TimeSeries(port_rets=port_rets, cs_weight=target_weight_df, call_method='kl', param=12).run()
cppi_weights, ew_cppi_weight = TimeSeries(port_rets=port_rets, cs_weight=target_weight_df, call_method='cppi', param=12).run()

In [28]:
ts_models = {'ew': ew_ew_weight,
            'vt': ew_vt_weight, 
            'cvt': ew_cvt_weight, 
            'kl': ew_kl_weight, 
            'cppi': ew_cppi_weight
            }

In [30]:
signal

,GOOGL,HD,IAU,IEF,INTU,KO,NOW,PEP,SHV,SLV,TLT,TSLA,UNH,VNQ
2013-06-28,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2013-07-31,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2013-08-30,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
2013-09-30,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2013-10-31,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
2022-10-31,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2022-11-30,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2022-12-30,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


In [35]:
price_df = add_cash(price=price_df, num_day_in_year=252, yearly_rfr=0.04)

In [39]:
for name, weight in ts_models.items():
    print(name)
    cum_rets = port_cum_rets(calculate_portvals(price_df=price_df, weight_df=weight, signal_df=signal, long_only=True))
    #display(Metric(cum_rets).rolling_metric()['dd'].plot())
    #plt.show()
    Metric(portfolio=cum_rets).print_report()
    print()

ew
Annualized Return: 12.41%
Annualized Volatility: 11.07%
Skewness: -0.21
Kurtosis: 11.92
Max Drawdown: -17.22%
Max Drawdown Duration: 494 days
Annualized Sharp Ratio: 0.76
Annualized Sortino Ratio: -0.31
Annualized Calmar Ratio: 0.72
Annualized VaR: -0.02
Annualized VaR Ratio: 0.03
Annualized CVaR: -0.03
Annualized CVaR Ratio: 0.02
Annualized hit Ratio: 0.56
Annualized GtP Ratio: 0.98

vt
Annualized Return: 10.80%
Annualized Volatility: 10.29%
Skewness: -0.65
Kurtosis: 17.15
Max Drawdown: -21.97%
Max Drawdown Duration: 393 days
Annualized Sharp Ratio: 0.66
Annualized Sortino Ratio: -0.34
Annualized Calmar Ratio: 0.49
Annualized VaR: -0.02
Annualized VaR Ratio: 0.02
Annualized CVaR: -0.03
Annualized CVaR Ratio: 0.01
Annualized hit Ratio: 0.60
Annualized GtP Ratio: 0.82

cvt
Annualized Return: 8.51%
Annualized Volatility: 67.51%
Skewness: 18.13
Kurtosis: 699.95
Max Drawdown: -92.01%
Max Drawdown Duration: 816 days
Annualized Sharp Ratio: 0.07
Annualized Sortino Ratio: -0.01
Annualized 